## pydantic -> 제일 많이 쓰는 형식

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser

class ReviewSummary(BaseModel):
    title : str
    bullets : list[str] = Field(description="주요 리뷰 사항", min_items = 3, max_items = 5)
    tone : str = Field(description="리뷰에 대한 평가")

parser = PydanticOutputParser(pydantic_object = ReviewSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewSummary'>)

In [6]:
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"items": {"type": "string"}, "maxItems": 5, "minItems": 3, "title": "Bullets", "type": "array"}, "tone": {"title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "JSON으로만 출력. \n\n 양식 : {fmt}"),
    ("user", "다음 텍스트를 의사 결정용으로 요약 :\n\n{text}")
]).partial(fmt=fmt)

In [8]:
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"items": {"type": "string"}, "maxItems": 5, "minItems": 3, "title": "Bullets", "type": "array"}, "tone": {"title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt'], input_types={}, partial_variables={}, template='JSON으로만 출력. \n\n 양식 : {fmt}'), additional_kwargs={}),

In [9]:
chain = prompt | model | parser

text = """
버튼 하나로 부드럽고 쫀쫀한 거품

디자인&컬러 깔끔하고 간편한 조작으로 사용하기는 쉽네요.
거품기 부품 스프링까지 쉽게 분리되어 세척이 간편하고 위생적이다.
우유는 눈금선까지 넣을 경우 100ml 사용된다.

사실 밀크프로더 사용 경험이 없어서 많이 설레였는데,
정말 단순.
중앙 버튼을 1초 누르면 따뜻한 거품,
3초 누르면 차가운 거품을 만들 수 있어요.

거품 만들기 시작을 누른 후에 자동 정지는 없고 적당한 선에서 다시 시작 버튼을 눌러줘야 되네요.

설명서가 초간단이라서 조금 아쉽네요.
시작 누르고 몇 초 후 정지를 눌러야 할지 반복적인 사용으로 터득해야 할 것 같네요.
거품은 쫀쫀하고 실키해요~bb

라떼고 자동 머신에서 카푸치노 만들어 마시다가
직접 만들었더니 조금 번거롭지만
요 이쁜놈 장식품되지 않게 자주 사용해보고
좋은 TIP 터득하면 한 달 후기 써볼까?
생각 중...

타브랜드에 비해 가격은 좀 비싸지만
성능이 중요하지!
디자인도 이쁨!

별 한 개 뺀 것은,
사용방법 설명이 자세하지 않고,
택배 걸 포장이 비닐이라서 안전 배송에 염려 됨.

가성비 성능에 비해 비싸요
성능 기대 이상 뛰어났어요
조작 편리성 조작이 직관적이에요
"""

In [16]:
try:
    result = chain.invoke({"text" : text})
    print(result)
except ValidationError as e:
    print("ValidationError: ", e)

title='밀크프로더 사용 후기 요약' bullets=['버튼 하나로 부드럽고 쫀쫀한 거품 생성 가능하며, 따뜻한 거품과 차가운 거품 선택 가능', '깔끔한 디자인과 간편한 조작, 부품 분리 및 세척이 쉬워 위생적임', '자동 정지 기능은 없으며, 사용법이 단순하지만 설명서가 부족해 반복 사용으로 익혀야 함', '가격은 다소 비싸지만 성능과 디자인이 뛰어나 만족스러움', '포장 안전성에 대한 우려와 설명서 부족으로 별 하나 감점'] tone='객관적이고 실용적인'


In [12]:
print(type(result))
# -> class 나옴

<class '__main__.ReviewSummary'>


In [13]:
# class 를 dict 로 바꾸기
result.model_dump()

{'title': '밀크프로더 사용 후기 요약',
 'bullets': ['버튼 하나로 부드럽고 쫀쫀한 거품 생성 가능, 중앙 버튼 1초 누르면 따뜻한 거품, 3초 누르면 차가운 거품',
  '깔끔한 디자인과 간편한 조작, 부품 분리 및 세척이 쉬워 위생적임',
  '자동 정지 기능 없음, 사용법이 단순하지만 설명서가 부족해 반복 사용으로 익혀야 함',
  '가격은 타 브랜드 대비 다소 높으나 성능과 디자인이 뛰어남',
  '포장 안전성에 대한 우려와 설명서 미흡으로 별 한 개 감점'],
 'tone': '객관적이고 실용적인 평가'}

## 실습

In [17]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser

class ReviewSummary(BaseModel):
    title : str
    bullets : list[str] = Field(description= "회의 내용", min_items = 5, max_items = 7)
    tone : str = Field(description = "회의 분위기")

parser = PydanticOutputParser(pydantic_object = ReviewSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewSummary'>)

In [20]:
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"items": {"type": "string"}, "maxItems": 7, "minItems": 5, "title": "Bullets", "type": "array"}, "tone": {"title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 회의록 작성 전문가야. 회의 내용을 보고 회의록 작성해줘. JSON으로만 출력. \n\n 양식 : {fmt}"),
    ("user", "다음 텍스트를 회의록 요약 : \n\n{text}")
]).partial(fmt=fmt)

In [22]:
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"items": {"type": "string"}, "maxItems": 7, "minItems": 5, "title": "Bullets", "type": "array"}, "tone": {"title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt'], input_types={}, partial_variables={}, template='당신은 회의록 작성 전문가야. 회의 내용을 보고 회의록 작성해줘. JSON으로만 출력. \n\n

In [23]:
chain = prompt | model | parser

text = """[회의 일시: 2025년 10월 1일 / 장소: 본사 3층 회의실]
참석자: 김대리, 이과장, 박팀장, 최이사

김대리: 지난달부터 진행 중인 SNS 광고 캠페인 결과 나왔습니다. 클릭률이 15% 정도 올랐어요.

이과장: 오, 그 정도면 꽤 잘 나온 거네. 인스타그램 인플루언서랑 진행한 것도 효과 있었나?

김대리: 네, 반응은 나쁘지 않았습니다. 다만 ROI는 아직 정확히 분석이 필요해요.

박팀장: 지금까지 전체 마케팅 예산의 60% 정도 집행했어요. 남은 예산으로 뭘 더 할 수 있을지 고민이 필요하네요.

최이사: 그래서 말인데, 11월부터는 영상 콘텐츠 쪽을 좀 더 강화해보는 게 어떨까 해요. 특히 브랜드 인지도 높이려면 짧은 영상이 좋을 것 같아서요.

이과장: 요즘 릴스나 유튜브 쇼츠 같은 포맷이 반응 좋죠. 우리도 그쪽으로 테스트 해보면 좋을 듯.

김대리: 영상 제작은 외주로 맡겨야 할 것 같아요. 제가 업체 세 군데 정도 견적 받아볼게요.

박팀장: 그리고 연말 고객 감사 이벤트도 슬슬 준비해야 해요. 작년엔 반응 좋았거든요.

최이사: 좋습니다. 그건 박팀장님이 총괄로 맡아주세요.

이과장: 사내 뉴스레터 11월호는 어떻게 하죠?

김대리: 아이디어 회의 일단 10월 8일로 잡아두겠습니다."""

In [24]:
try:
    result = chain.invoke({"text" : text})
    print(result)
except ValidationError as e:
    print("ValidationError: ", e)

title='2025년 10월 1일 마케팅 전략 회의' bullets=['SNS 광고 캠페인 클릭률 15% 상승, 인스타그램 인플루언서 협업 효과 긍정적이나 ROI 추가 분석 필요', '전체 마케팅 예산의 60% 집행 완료, 남은 예산 활용 방안 모색 중', '11월부터 영상 콘텐츠 강화 계획, 짧은 영상(릴스, 유튜브 쇼츠) 중심으로 브랜드 인지도 제고 목표', '영상 제작 외주 예정, 김대리가 3개 업체 견적 수집 담당', '연말 고객 감사 이벤트 준비 필요, 박팀장이 총괄 담당', '사내 뉴스레터 11월호 아이디어 회의 10월 8일 예정'] tone='전문적이고 협력적인'
